In [122]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd
import lxml.html
from ast import literal_eval
import random

In [2]:
url = 'https://www.netpincer.hu/restaurants/new?lat=46.2576495&lng=20.1505831&vertical=restaurants'

In [3]:
driver = webdriver.Chrome()
driver.get(url=url)
time.sleep(2)

closed_section = driver.find_elements_by_css_selector("[class='vendor-list-section']")

while True:
    if not closed_section:
        driver.execute_script("a = document.getElementsByClassName('vendor-list-section open-section');")
        driver.execute_script("window.scrollTo(0, a[0].scrollHeight);")
        driver.implicitly_wait(5)
        closed_section = driver.find_elements_by_css_selector("[class='vendor-list-section']")
    else:
        break

src = driver.page_source
driver.quit()

In [5]:
soup = BeautifulSoup(src, 'html.parser')

In [74]:
lista = soup.find('section', {'class': 'vendor-list-section open-section'}).find('ul', {'class': 'vendor-list'}).findChildren(recursive=False)

In [108]:
# Nev, kategoriak, ertekeles, link, szallitasi dij
restaurants = pd.DataFrame(columns=['name', 'rating', 'link', 'categories', 'delivery_fee'])
for item in lista:
    rest_name = item.find('span', {'class': 'name fn'}).text.strip()
    try:
        rating = item.find('span', {'class': 'rating'}).b.text.strip()
    except:
        rating = 'n/a'
    link = f"https://www.netpincer.hu{item.a['href']}"
    categories = [a.text.strip() for a in item.find('li', {'class': 'vendor-characteristic'}).findAll('span')]
    try:
        delivery_fee = item.find('span', {'data-cy': 'delivery-fee-active'}).strong.text.strip()
    except:
        delivery_fee = '0 Ft'
    restaurants = restaurants.append(pd.Series({'name': rest_name, 
                                  'rating': rating, 
                                  'link': link, 
                                  'categories':categories, 
                                  'delivery_fee': delivery_fee}), ignore_index=True)


In [114]:
del_cats = ['Allergén', 'Algida', 'Alkoholos italok', 'Cukrászda', 'Fagyi', 'Italok', 'Jótékony', 'Kedvezmények 15%-tól',
'Készpénzes fizetés', 'NetPincér GO', 'SZÉP kártyás fizetés', 'Vegetariánus', 'Érintkezésmentes kiszállítás', 'Fusion']

restaurants['categories'] = restaurants.categories.apply(lambda x: [a for a in x if a not in del_cats])

restaurants= restaurants[restaurants['categories'].map(lambda d: len(d)) > 0]

def converter(x):
    return literal_eval(x)
converters={'categories': converter}

In [115]:
cats = []
for a in restaurants.categories:
    cats.extend(a)
cats = sorted(list(set(cats)))

In [116]:
cats

['Amerikai',
 'Grill',
 'Gyorsétterem',
 'Görög',
 'Halétel',
 'Hamburger',
 'Indiai',
 'Kínai',
 'Különlegességek',
 'Leves',
 'Magyaros',
 'Mexikói',
 'Napi menü',
 'Nemzetközi',
 'Olasz',
 'Pizza',
 'Pékség',
 'Reggelizőhely',
 'Saláta',
 'Spanyol',
 'Szendvics',
 'Szerb',
 'Tészta',
 'Török',
 'Vegetáriánus',
 'Ázsiai',
 'Édességek',
 'Ételérzékenyeknek is']

In [117]:
restaurants

,name,rating,link,categories,delivery_fee
0,Inferno Burgers,4.2,https://www.netpincer.hu/restaurant/y1yv/infer...,"[Hamburger, Amerikai]",470 Ft
1,Élesztő Pince,3.7,https://www.netpincer.hu/restaurant/oag5/elesz...,[Pizza],470 Ft
2,"McDonald's - Szeged, Kárász utca",4.6,https://www.netpincer.hu/restaurant/g4yx/mcdon...,"[Hamburger, Szendvics, Gyorsétterem, Ételérzék...",590 Ft
3,Pizzaguru,4.7,https://www.netpincer.hu/restaurant/i9um/pizza...,"[Pizza, Saláta, Leves, Tészta, Leves, Saláta, ...",390 Ft
4,Blues Kert Vendéglő,4.4,https://www.netpincer.hu/restaurant/d0kv/blues...,"[Magyaros, Amerikai]",490 Ft
...,...,...,...,...,...
136,Boss Pizzéria és Hamburger Bár,5,https://www.netpincer.hu/restaurant/s0bt/boss-...,[Pizza],900 Ft
137,Szélmalom Vendéglő,5,https://www.netpincer.hu/restaurant/e8de/szelm...,[Napi menü],690 Ft
138,Temesy Hidegkonyha,n/a,https://www.netpincer.hu/restaurant/uw9e/temes...,[Szendvics],1 200 Ft
139,Befalom Bistro,5,https://www.netpincer.hu/restaurant/r0gm/befal...,[Magyaros],290 Ft


In [131]:
mask = restaurants.categories.apply(lambda x: 'Magyaros' in x)

restaurants[mask]

,name,rating,link,categories,delivery_fee
4,Blues Kert Vendéglő,4.4,https://www.netpincer.hu/restaurant/d0kv/blues...,"[Magyaros, Amerikai]",490 Ft
13,PipiPont,4.9,https://www.netpincer.hu/restaurant/w7uh/pipipont,"[Szendvics, Magyaros]",290 Ft
15,Gulyás Csárda,4.6,https://www.netpincer.hu/restaurant/l1bd/gulya...,[Magyaros],290 Ft
21,Prézli Szeged,4.9,https://www.netpincer.hu/restaurant/s9ci/prezl...,"[Magyaros, Ételérzékenyeknek is]",290 Ft
32,John Bull Pub,4.8,https://www.netpincer.hu/restaurant/d9ql/john-...,"[Magyaros, Amerikai]",400 Ft
33,Magyaros Ételbár,4.2,https://www.netpincer.hu/restaurant/u6zi/magya...,"[Magyaros, Napi menü]",290 Ft
35,Kálmán Panzió és Étterem,4.7,https://www.netpincer.hu/restaurant/p3nh/kalma...,[Magyaros],290 Ft
47,HBH Étterem,4.7,https://www.netpincer.hu/restaurant/r1rz/hbh-e...,[Magyaros],290 Ft
48,Bringa Bisztró,4.6,https://www.netpincer.hu/restaurant/i5un/bring...,[Magyaros],290 Ft
49,Coop Étterem,4.7,https://www.netpincer.hu/restaurant/u3lp/coop-...,[Magyaros],290 Ft
